In [97]:
df <- read.csv("training_data.csv", stringsAsFactors=T, header = TRUE, strip.white = TRUE, sep = ",")
#df <- df[, setdiff(names(df), "X")]
#df_labels <- read.csv("training_labels.csv", header = TRUE, strip.white = TRUE, sep = ",")

In [98]:
library(rlang)
library(dummies)
library(stringdist)
library(dplyr)
library(randomForest)

In [99]:
lowercase_all = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "factor") {
            df[,column] <<- tolower(df[,column])
        }
    }
}

In [100]:
lowercase_all(df)

In [101]:
#filter long/lat
x_up_right = 0.022
y_up_right = 40.729
x_down_left = -12.729
y_down_left = 28.138

df <- df[df$latitude > x_down_left & df$latitude < x_up_right & df$longitude > y_down_left & df$longitude < y_up_right, ] #filter pumps in atlantic fucking ocean

In [102]:
drops <- c("latitude", "longitude", "funder", "date_recorded", "gps_height", "wpt_name", "num_private", "lga", "ward", "subvillage", "region_code", "district_code", "recorded_by", "scheme_name", "waterpoint_type_group", "payment", "management", "management_group", "extraction_type_group", "extraction_type", "quantity_group", "quality_group", "amount_tsh", "population", "public_meeting", "scheme_management", "permit", "source_type", "source_class")
df = df[ , !(names(df) %in% drops)] #drop useless (a priori) features, maybe after feature selection remove less

In [103]:
df

,id,installer,basin,region,construction_year,extraction_type_class,payment_type,water_quality,quantity,source,waterpoint_type
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,roman,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,grumeti,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,world vision,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,unicef,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
5,19728,artisan,lake victoria,kagera,0,gravity,never pay,soft,seasonal,rainwater harvesting,communal standpipe
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
7,19816,dwsp,internal,shinyanga,0,handpump,never pay,soft,enough,machine dbh,hand pump
8,54551,dwe,lake tanganyika,shinyanga,0,handpump,unknown,milky,enough,shallow well,hand pump
9,53934,water aid,lake tanganyika,tabora,0,handpump,never pay,salty,seasonal,machine dbh,hand pump


In [38]:
work_on = setdiff(names(df), c("id"))
df[work_on][df[work_on] == 0 | df[work_on] == "-" | df[work_on] == "" | df[work_on] == "unknown"] = NA

In [39]:
sum(complete.cases(df)) #check how many rows contain at least one NA val
full_df <- na.omit(df)

[1] 31383

In [40]:
na_df = dplyr::setdiff(df,full_df)
dim(na_df)

[1] 26205    11

In [11]:
similarity_test_apply = function(line) {
    NAs = is.na(line)
    idxs = which(is.na(line))
            #print(df[names(df)[idxs]])
    line2 = line[-idxs]
    most_similar = full_df[which.min(Reduce(`+`,Map(stringdist, setdiff(line2, full_df[, -idxs]), line2, method='jaccard'))),]

    line[idxs] <- most_similar[idxs] 
    to_change = which(strtoi(df$id) == strtoi(line$id))
    assign('df',df,envir=.GlobalEnv)
    df[to_change, idxs] <<- most_similar[idxs]
        
}

In [12]:
#testing similarity

start.time = Sys.time()
res = apply(na_df[1:30,], 1, similarity_test_apply)
round(Sys.time() - start.time,5)

Time difference of 28.26917 secs

In [41]:
indx <- tail(names(sort(table(df$installer))),10)
print(indx)
to_change = which(!(full_df$installer %in% indx))
full_df[to_change, 'installer'] = 'other'
full_df

 [1] "tcrs"               "central government" "kkkt"              
 [4] "district council"   "danida"             "commu"             
 [7] "rwe"                "hesawa"             "government"        
[10] "dwe"               


,id,installer,basin,region,construction_year,extraction_type_class,payment_type,water_quality,quantity,source,waterpoint_type
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,other,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,other,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,other,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,other,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
11,49056,other,wami / ruvu,pwani,2011,submersible,never pay,salty,enough,machine dbh,other
12,50409,danida,lake nyasa,ruvuma,1987,handpump,on failure,soft,insufficient,shallow well,hand pump
14,50495,other,pangani,kilimanjaro,2009,gravity,monthly,soft,enough,spring,communal standpipe
16,61848,dwe,lake tanganyika,rukwa,1991,handpump,never pay,soft,enough,machine dbh,hand pump


In [23]:
df.pca <- prcomp(dftest_, center = TRUE,scale. = TRUE)

In [27]:
summary(df.pca)

Importance of components:
                           PC1     PC2     PC3     PC4     PC5     PC6     PC7
Standard deviation     2.31248 1.86417 1.79532 1.62256 1.61976 1.60173 1.51008
Proportion of Variance 0.06218 0.04041 0.03748 0.03061 0.03051 0.02983 0.02652
Cumulative Proportion  0.06218 0.10259 0.14007 0.17068 0.20119 0.23102 0.25754
                           PC8     PC9    PC10    PC11    PC12    PC13    PC14
Standard deviation     1.48260 1.46871 1.38214 1.36365 1.31759 1.30277 1.27072
Proportion of Variance 0.02556 0.02508 0.02221 0.02162 0.02019 0.01974 0.01878
Cumulative Proportion  0.28310 0.30818 0.33039 0.35201 0.37220 0.39194 0.41071
                          PC15   PC16    PC17    PC18   PC19    PC20    PC21
Standard deviation     1.21304 1.1948 1.17903 1.15984 1.1435 1.12950 1.11702
Proportion of Variance 0.01711 0.0166 0.01616 0.01564 0.0152 0.01483 0.01451
Cumulative Proportion  0.42782 0.4444 0.46058 0.47623 0.4914 0.50627 0.52077
                          PC22    

In [43]:
df_labels <- read.csv("training_labels.csv", stringsAsFactors=T, header = TRUE, strip.white = TRUE, sep = ",")

In [112]:
Merged_df = full_df

In [113]:
Merged_df<-merge(Merged_df,df_labels,by="id",all=FALSE)
dim(Merged_df)
Merged_df = Merged_df[,2:ncol(Merged_df)]
dim(Merged_df)

[1] 31383     8

[1] 31383     7

In [114]:
Merged_df = dummy.data.frame(Merged_df, sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"


In [69]:
dim(Merged_df)

[1] 31383    80

In [47]:
Merged_df[,70:81]

,source_shallow well,source_spring,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group_functional,status_group_functional needs repair,status_group_non functional
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,0,0,0,0,0,1,0,0,1,0,0
2,0,1,0,1,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,1
4,0,1,0,1,0,0,0,0,0,0,0,1
5,0,1,0,1,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0,1,0,0
7,0,0,0,0,1,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,1,0,0,1
9,0,1,0,1,0,0,0,0,0,0,0,1


In [110]:
#Target var not one hot encod.
library("randomForest")
n_trees <- 20
accuracy_vec <- array(0,n_trees)

df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]
    
model_rf <- randomForest(x=train_data[,-c(target_variable)],
                         y=as.factor(train_data[,c(target_variable)]),
                         xtest=test_data[,-c(target_variable)],
                         ytest=as.factor(test_data[,c(target_variable)]),
                         ntree=n_trees, do.trace=T)
    

accuracy_vec[0] = (model_rf$test$confusion[1,1]+model_rf$test$confusion[2,2])/sum(model_rf$test$confusion)

#plot(accuracy_vec,main = "Number of trees influence",xlab = "Nbr of trees",ylab = "Classification rate") 

ntree      OOB      1      2      3|    Test      1      2      3
    1:  27.49%  6.35% 85.23% 52.72%|  28.00%  6.50% 84.06% 54.31%
    2:  26.77%  7.42% 89.05% 47.99%|  28.10%  6.52% 89.64% 53.54%
    3:  26.98%  7.60% 92.06% 47.43%|  26.08%  5.57% 93.82% 48.53%
    4:  26.65%  7.71% 92.67% 46.23%|  25.52%  4.99% 94.62% 47.75%
    5:  26.24%  7.55% 91.45% 45.61%|  25.31%  5.66% 95.02% 45.95%
    6:  26.20%  7.75% 89.52% 45.49%|  25.29%  5.51% 91.83% 46.72%
    7:  26.14%  7.66% 90.88% 45.17%|  25.22%  6.37% 94.62% 44.55%
    8:  25.76%  7.59% 88.38% 44.69%|  25.07%  6.50% 90.64% 44.62%
    9:  25.54%  7.44% 89.36% 44.09%|  25.11%  6.54% 93.43% 44.14%
   10:  25.43%  7.12% 88.96% 44.38%|  24.92%  6.37% 89.44% 44.62%
   11:  25.33%  6.98% 89.73% 44.16%|  25.21%  6.31% 92.83% 44.95%
   12:  25.14%  6.72% 90.00% 44.02%|  25.06%  5.79% 92.23% 45.50%
   13:  25.01%  6.48% 88.71% 44.32%|  24.97%  5.87% 92.43% 45.06%
   14:  25.02%  6.40% 88.59% 44.54%|  25.10%  6.00% 90.64% 45.54%
   15:  24

In [95]:
y_pred_rf = predict(model_rf, newdata = test_data[,-target_variable])

ERROR: Error in predict.randomForest(model_rf, newdata = test_data[, -target_variable]): No forest component in the object


In [111]:
print((model_rf$test$confusion[1,1]+model_rf$test$confusion[2,2])/sum(model_rf$test$confusion))

[1] 0.5642666


In [48]:
# Neural network
library(nnet)

In [119]:

df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]

hidden_nodes<-72
threshold<-0.5
target_variable <-c(ncol(Merged_df)-2,ncol(Merged_df)-1,ncol(Merged_df))

model_nn<-nnet(x=train_data[,-target_variable],
               y=train_data[,target_variable],
               size=hidden_nodes,
               softmax=T,
               skip=FALSE,
               trace=T, 
               maxit=1000,
               MaxNWts=100000,
               rang=0.5)

model_nn

Y_pred<-predict(model_nn,test_data[,-target_variable])




#Y_hat_one_hot <- ifelse(Y_pred[,2] > threshold,"functional","functional needs repair")

head(Y_pred)

# weights:  3315
initial  value 20517.025433 
iter  10 value 20254.408500
iter  20 value 19756.954105
iter  30 value 19594.748049
iter  40 value 19538.749178
iter  50 value 19349.303247
iter  60 value 18774.264181
iter  70 value 17637.367081
iter  80 value 17031.289144
iter  90 value 16738.004999
iter 100 value 16620.010108
iter 110 value 16612.075615
final  value 16611.126934 
converged


a 42-72-3 network with 3315 weights
options were - softmax modelling 

,status_group_functional,status_group_functional needs repair,status_group_non functional
9795,0.5785278,0.08385962,0.3376126
20452,0.7242713,0.06798377,0.2077450
29187,0.7411308,0.06562368,0.1932455
4191,0.7870069,0.05856137,0.1544317
22099,0.6578322,0.07618163,0.2659862
31001,0.7343267,0.06659060,0.1990827


In [120]:
y_hat_idx = cbind(1:nrow(Y_pred), max.col(Y_pred, 'first'))

In [121]:
Y <- test_data[,target_variable]
y_idx = cbind(1:nrow(Y), max.col(Y, 'first'))

In [122]:
good_preds = which(y_hat_idx[,2] == y_idx[,2])
n_good = length(good_preds)
print(n_good/nrow(Y))

[1] 0.7303084


In [124]:
### SVM ###
library(e1071)

Warning message:
"package 'e1071' was built under R version 4.0.5"


In [160]:
df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable1 <-ncol(Merged_df)-2
target_variable2 <-ncol(Merged_df)-1 
target_variable3 <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]

train_funct = train_data[,-which(names(train_data) == "status_group_functional needs repair" | names(train_data) == "status_group_non functional")]
train_funct_rep = train_data[,-which(names(train_data) == "status_group_functional" | names(train_data) == "status_group_non functional")]
train_non_funct = train_data[,-which(names(train_data) == "status_group_functional needs repair" | names(train_data) == "status_group_functional")]

test_funct = test_data[,-which(names(test_data) == "status_group_functional needs repair" | names(train_data) == "status_group_non functional")]
test_funct_rep = test_data[,-which(names(test_data) == "status_group_functional" | names(train_data) == "status_group_non functional")]
test_non_funct = test_data[,-which(names(test_data) == "status_group_functional needs repair" | names(train_data) == "status_group_functional")]

DS<-cbind(train_funct,functional = train_funct[,ncol(train_funct)])

# Model fit (using lm function)
model_svm_f<- svm(functional~.,DS)

# Model prediction 
Y_hat_f<- predict(model_svm_f,test_funct)

DS<-cbind(train_funct_rep,funct_rep = train_funct_rep[,ncol(train_funct_rep)])

# Model fit (using lm function)
model_svm_fr<- svm(funct_rep~.,DS)

# Model prediction 
Y_hat_fr<- predict(model_svm_fr,test_funct_rep)

DS<-cbind(train_non_funct,non_functional = train_non_funct[,ncol(train_non_funct)])

# Model fit (using lm function)
model_svm_nf<- svm(non_functional~.,DS)

# Model prediction 
Y_hat_nf<- predict(model_svm_nf,test_non_funct)


In [168]:
Merged_df

,region_arusha,region_iringa,region_kigoma,region_kilimanjaro,region_lindi,region_manyara,region_mara,region_morogoro,region_mtwara,region_mwanza,...,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group_functional,status_group_functional needs repair,status_group_non functional
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
5,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
7,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [174]:
Y_hat_fr = data.frame(Y_hat_fr)
Y_hat_nf = data.frame(Y_hat_nf)
Y_hat_f = data.frame(Y_hat_f)
test = cbind(Y_hat_fr, Y_hat_nf, Y_hat_f)
test

,Y_hat_fr,Y_hat_nf,Y_hat_f
,<dbl>,<dbl>,<dbl>
24830,1.010465464,0.023310808,0.011734079
5357,0.017339880,0.952481799,0.049039601
19873,0.024597839,0.045299681,0.957125850
25306,0.020568189,0.955443138,0.046757458
254,0.025042503,0.952640674,0.049020479
28237,1.023275687,-0.040448654,-0.043042330
7850,0.002204165,1.010336486,-0.002733843
307,0.017588209,0.002883009,1.004544157
9462,-0.018617531,-0.034261964,1.035430819


In [176]:
y_hat_idx_svm = cbind(1:nrow(test), max.col(test, 'first'))
y_hat_idx_svm

1,1
2,2
3,3
4,2
5,2
6,1
7,2
8,3
9,3
10,3
11,3


In [ ]:
Y <- test_data[,ncol(test_data)-2:ncol(test_data)]
Y

In [ ]:
good_preds = which(y_hat_idx[,2] == y_idx[,2])
n_good = length(good_preds)
print(n_good/nrow(Y))

In [150]:
colnames(Y_hat_f) <- c("funct")
colnames(Y_hat_nf) <- c("non functional")
colnames(Y_hat_fr) <- c("funct needs rep")

In [171]:
test_funct

,region_arusha,region_iringa,region_kigoma,region_kilimanjaro,region_lindi,region_manyara,region_mara,region_morogoro,region_mtwara,region_mwanza,...,quantity_insufficient,quantity_seasonal,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group_functional
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
24830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5357,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
19873,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
25306,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
254,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
28237,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
7850,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
307,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
9462,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1


In [156]:
Y_hat_fr

,funct needs rep
,<dbl>
15957,-0.025230851
17286,0.020969155
251,0.026229415
18273,-0.019891843
23026,-0.025085107
17046,0.020347153
22888,-0.025312535
24135,-0.022485714
19034,-0.003776749


In [ ]:
###TESTING FEATURES


In [105]:
drops <- c("installer", "basin", "source", "payment_type")
full_df = full_df[ , !(names(full_df) %in% drops)] #drop useless (a priori) features, maybe after feature selection remove less

In [106]:
full_df

,id,region,construction_year,extraction_type_class,water_quality,quantity,waterpoint_type
,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,69572,iringa,1999,gravity,soft,enough,communal standpipe
2,8776,mara,2010,gravity,soft,insufficient,communal standpipe
3,34310,manyara,2009,gravity,soft,enough,communal standpipe multiple
4,67743,mtwara,1986,submersible,soft,dry,communal standpipe multiple
6,9944,tanga,2009,submersible,salty,enough,communal standpipe multiple
11,49056,pwani,2011,submersible,salty,enough,other
12,50409,ruvuma,1987,handpump,soft,insufficient,hand pump
14,50495,kilimanjaro,2009,gravity,soft,enough,communal standpipe
16,61848,rukwa,1991,handpump,soft,enough,hand pump


In [15]:
indx <- tail(names(sort(table(full_df$extraction_type_class))),17)
print(indx)
to_change = which(!(full_df$extraction_type %in% indx))
full_df[to_change, 'extraction_type'] = 'other'
full_df

 [1] "other - mkulima/shinyanga" "walimi"                   
 [3] "other - play pump"         "windmill"                 
 [5] "india mark iii"            "other - swn 81"           
 [7] "other - rope pump"         "afridev"                  
 [9] "ksb"                       "mono"                     
[11] "india mark ii"             "swn 80"                   
[13] "nira/tanira"               "other"                    
[15] "submersible"               "gravity"                  


,id,installer,basin,region,construction_year,extraction_type,payment_type,water_quality,quantity,source,waterpoint_type
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,other,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,other,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,other,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,other,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
11,49056,other,wami / ruvu,pwani,2011,submersible,never pay,salty,enough,machine dbh,other
12,50409,danida,lake nyasa,ruvuma,1987,swn 80,on failure,soft,insufficient,shallow well,hand pump
14,50495,other,pangani,kilimanjaro,2009,gravity,monthly,soft,enough,spring,communal standpipe
16,61848,dwe,lake tanganyika,rukwa,1991,swn 80,never pay,soft,enough,machine dbh,hand pump
